Proyecto Final de Redes Neuronales Artificiales
"Confiabilidad de Productos en Amazon Basado en reseñas"

Equipo 7:
Karime Kirey Sánchez Andrade 1914174
Lázaro Gael Cortez Moncada



---



PREPROCESAMIENTO DE LOS DATOS
---



In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

# el dataset puede obtenerse del siguiente link https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt
# Importamos el dataset utilizado contiene reseñas de artículos de la categoría de Oficina. Especificamos que los encabezados están en la fila 0

dtf = pd.read_table('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', header=0, on_bad_lines='skip')
df=pd.DataFrame(dtf) #Convertimos el dataset en un Dataframe con pandas
print(df.head())

df.shape
df.info()

<ipython-input-188-68fea294cbff>:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dtf = pd.read_table('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', header=0, on_bad_lines='skip')


  marketplace  customer_id       review_id  product_id  product_parent  \
0          US     43081963  R18RVCKGH1SSI9  B001BM2MAC       307809868   
1          US     10951564  R3L4L6LW1PUOFY  B00DZYEXPQ        75004341   
2          US     21143145  R2J8AWXWTDX2TF  B00RTMUHDW       529689027   
3          US     52782374  R1PR37BR7G3M6A  B00D7H8XB6       868449945   
4          US     24045652  R3BDDDZMZBZDPU  B001XCWP34        33521401   

                                       product_title product_category  \
0     Scotch Cushion Wrap 7961, 12 Inches x 100 Feet  Office Products   
1          Dust-Off Compressed Gas Duster, Pack of 4  Office Products   
2  Amram Tagger Standard Tag Attaching Tagging Gu...  Office Products   
3  AmazonBasics 12-Sheet High-Security Micro-Cut ...  Office Products   
4  Derwent Colored Pencils, Inktense Ink Pencils,...  Office Products   

  star_rating  helpful_votes  total_votes vine verified_purchase  \
0           5            0.0          0.0    N  

In [ ]:
df['review_date'] = pd.to_datetime(df['review_date'])
reviews_2015 = df[df['review_date'].dt.year == 2015]
count_reviews_2015 = len(reviews_2015)
print("El número de reseñas de 2015 es: ", count_reviews_2015)

#Eliminamos todos los datos que no sean del 2015
datos = df[df['review_date'].dt.year == 2015]
datos.shape

El número de reseñas de 2015 es:  815817


(815817, 15)

In [ ]:
#Eliminaremos filas (aleatoriamente) para quedarnos únicamente con 50000 datos del 2015
datos = datos.sample(n=50000, random_state=42)
datos.shape

(50000, 15)

In [ ]:
#Eliminación de columnas que no se utilizarán con la función de pandas "drop"
columnas_quitar = ['marketplace', 'customer_id', 'review_id', 'product_parent', 'vine']
datos=datos.drop(columnas_quitar, axis=1)

print(datos.head(10))
datos.shape

        product_id                                      product_title  \
115271  B000VKSLPU  LD &copy; Remanufactured Replacement Laser Ton...   
61451   B00OGARPS0                      Drimark AC Adaptor (351TRIAD)   
261883  B000UTO0XO   Lamy Safari Fountain Pen, Yellow Fine Nib (L18F)   
51718   B001EY0A9C  Cool Toner 4 Pack Compatible Canon 104 Cartrid...   
176180  B00XTOQO9K  Powemall 2.4V,600mAh,Ni-MH, 3 Packs Cordless H...   
792026  B00BI5DYWW  Dooni Designs Monogram Initial Designs - Regal...   
299284  B0006NDRXS                      SHOCKING PEN - EMPIRE EDITION   
654322  B00EHDZMGI  Canon PIXMA Printing Solutions MG5520 Wireless...   
37340   B003IBLING  Pendaflex Hanging File Folder Tabs, 1/3 Tab,25...   
298849  B0094J7QW8  Inktec Brand Inkjet Refill Kit for Canon Pg-24...   

       product_category star_rating  helpful_votes  total_votes  \
115271  Office Products           5            0.0          0.0   
61451   Office Products           1            1.0          3.

(50000, 10)

In [ ]:
#Limpieza y preprocesamiento de texto: eliminación de puntuación, mayúsculas, stopwords

import re
import string
from gensim.parsing.preprocessing import remove_stopwords

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = remove_stopwords(text)
    return text

In [ ]:
datos['review_body'] = datos['review_body'].fillna('').astype(str) #algunas reseñas contienen valores numéricos. Se convierte todo a str y elimina números
datos['review_body'] = datos['review_body'].apply(clean_text)

In [ ]:
mapping = {'Y': 1, 'N': 0}  #Convertir el Y/N en 0s y 1s
datos['verified_purchase'] = datos['verified_purchase'].map(mapping)

In [ ]:
datos = datos.iloc[1:, :]
from IPython.display import display
display(datos.head(10))
print("El dataframe datos tiene el review_body limpio, sin signos de puntuación ni mayúsculas")

,product_id,product_title,product_category,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_date
61451,B00OGARPS0,Drimark AC Adaptor (351TRIAD),Office Products,1,1.0,3.0,1,One Star,lasted day light work,2015-08-14
261883,B000UTO0XO,"Lamy Safari Fountain Pen, Yellow Fine Nib (L18F)",Office Products,5,1.0,1.0,1,I like the Safari series by Lamy,like safari series lamy writes wish send nib e...,2015-06-13
51718,B001EY0A9C,Cool Toner 4 Pack Compatible Canon 104 Cartrid...,Office Products,4,0.0,0.0,1,Four Stars,good price fast shipping,2015-08-17
176180,B00XTOQO9K,"Powemall 2.4V,600mAh,Ni-MH, 3 Packs Cordless H...",Office Products,4,2.0,3.0,1,Great price and,batteries work advertised delivered time great...,2015-07-11
792026,B00BI5DYWW,Dooni Designs Monogram Initial Designs - Regal...,Office Products,1,0.0,0.0,1,Terrible quality,home bad quality,2015-01-06
299284,B0006NDRXS,SHOCKING PEN - EMPIRE EDITION,Office Products,1,0.0,0.0,1,One Star,ok,2015-06-02
654322,B00EHDZMGI,Canon PIXMA Printing Solutions MG5520 Wireless...,Office Products,4,0.0,0.0,1,I do wish they would have made it clear that o...,little slow compared laser printer price compl...,2015-02-16
37340,B003IBLING,"Pendaflex Hanging File Folder Tabs, 1/3 Tab,25...",Office Products,5,0.0,0.0,1,Five Stars,arrvied time expected,2015-08-21
298849,B0094J7QW8,Inktec Brand Inkjet Refill Kit for Canon Pg-24...,Office Products,2,0.0,0.0,1,Two Stars,worked refill work second time ran bummed,2015-06-02
771477,B00LO5N4ZS,Philips DVT6500 Voice Tracer Digital Recorder ...,Office Products,5,10.0,11.0,1,A Very Great Recorder!,recorder greatbr br ordered recorder based rea...,2015-01-11


El dataframe datos tiene el review_body limpio, sin signos de puntuación ni mayúsculas


In [ ]:
#FILTRAR PARA VER EJEMPLO DE RESEÑAS DE UN PRODUCTO

filtered_data = datos.loc[df['product_title'] == 'Lamy Safari Fountain Pen, Yellow Fine Nib (L18F)']

# Imprimir los datos filtrados
print(filtered_data)

        product_id                                     product_title  \
261883  B000UTO0XO  Lamy Safari Fountain Pen, Yellow Fine Nib (L18F)   
207251  B000UTO0XO  Lamy Safari Fountain Pen, Yellow Fine Nib (L18F)   

       product_category star_rating  helpful_votes  total_votes  \
261883  Office Products           5            1.0          1.0   
207251  Office Products           4            0.0          0.0   

        verified_purchase                                  review_headline  \
261883                  1                 I like the Safari series by Lamy   
207251                  1  Works. The nib is smooth writing. Pen feels ...   

                                              review_body review_date  
261883  like safari series lamy writes wish send nib e...  2015-06-13  
207251      works nib smooth writing pen feels solid hand  2015-07-01  


# ETIQUETACIÓN DE LOS DATOS: POLARIZACIÓN DE RESEÑAS

In [ ]:
#ETIQUETACIÓN DE DATOS: POSITIVO, NEGATIVO, NEUTRAL Y SCORE DE CADA UNO
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# Crea una instancia del analizador de sentimientos
sia = SentimentIntensityAnalyzer()

# Define una oración para analizar
df_sentiment = pd.DataFrame(datos['review_body'])

def obtener_sentimiento(reseña):
    scores = sia.polarity_scores(reseña)
    sentimiento = 'Positivo' if scores['compound'] > 0 else 'Negativo' if scores['compound'] < 0 else 'Neutral'
    positividad = scores['compound']
    return sentimiento, positividad


# Calificar el sentimiento de todas las reseñas y agregar las columnas correspondientes
df_sentiment[['sentimiento', 'positividad']] = df_sentiment['review_body'].apply(lambda x: pd.Series(obtener_sentimiento(x)))

# Imprimir el DataFrame con las columnas de sentimiento y positividad
print(df_sentiment)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                              review_body sentimiento  \
61451                               lasted day light work     Neutral   
261883  like safari series lamy writes wish send nib e...    Positivo   
51718                            good price fast shipping    Positivo   
176180  batteries work advertised delivered time great...    Positivo   
792026                                   home bad quality    Negativo   
...                                                   ...         ...   
455241                             genuine works problems    Negativo   
42200                                                  ok    Positivo   
505012  yes chord phone portable best worlds loud clea...    Positivo   
722140     bought staples acer touchscreen warranty sucks    Negativo   
135850                                             flimsy     Neutral   

        positividad  
61451        0.0000  
261883       0.9273  
51718        0.4404  
176180       0.6249  
792026      -

In [ ]:
display(df_sentiment.head(20))

,review_body,sentimiento,positividad
61451,lasted day light work,Neutral,0.0000
261883,like safari series lamy writes wish send nib e...,Positivo,0.9273
51718,good price fast shipping,Positivo,0.4404
176180,batteries work advertised delivered time great...,Positivo,0.6249
792026,home bad quality,Negativo,-0.5423
299284,ok,Positivo,0.2960
654322,little slow compared laser printer price compl...,Positivo,0.7184
37340,arrvied time expected,Neutral,0.0000
298849,worked refill work second time ran bummed,Neutral,0.0000
771477,recorder greatbr br ordered recorder based rea...,Positivo,0.9870


In [ ]:
#Se asignan valores numéricos a los sentimientos: 1-Positivo, -1-Negativo, 0-Neutral
df_sent_prep = df_sentiment.copy()
df_sent_prep['sentimiento'] = df_sent_prep['sentimiento'].map({'Negativo': -1, 'Positivo': 1, 'Neutral':0})
display(df_sent_prep.head(100))

,review_body,sentimiento,positividad
61451,lasted day light work,0,0.0000
261883,like safari series lamy writes wish send nib e...,1,0.9273
51718,good price fast shipping,1,0.4404
176180,batteries work advertised delivered time great...,1,0.6249
792026,home bad quality,-1,-0.5423
...,...,...,...
200414,fun useful happy,1,0.8720
574772,beautiful writing instrument beat price,1,0.5994
25047,great price fast delivery,1,0.6249
298868,thanks order future,1,0.4404


In [ ]:
#Se crea una copia del dataframe
df_sentiment_num = df_sent_prep.copy()

In [ ]:
#Importación de librerías
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#FILTRAR PARA VERIFICAR LAS RESEÑAS NEGATIVAS

filtered_sentiment = df_sent_prep.loc[df_sent_prep['positividad'] < 0]
# Imprimir los datos filtrados
print(filtered_sentiment)

                                              review_body  sentimiento  \
792026                                   home bad quality           -1   
77999   disappointed color staples peeled case staples...           -1   
563183                                     died worthless           -1   
613139                       sharpie doesnt love sharpies           -1   
307635              stops printing reason lots paper jams           -1   
...                                                   ...          ...   
735483  noisy old brother machine quiet comparison mis...           -1   
2348    sure problem lies lexmark printer rejects cart...           -1   
144366  advertise replaced old batteries panasonic pho...           -1   
455241                             genuine works problems           -1   
722140     bought staples acer touchscreen warranty sucks           -1   

        positividad  
792026      -0.5423  
77999       -0.0772  
563183      -0.7579  
613139      -0.5216  
3

In [ ]:
set_etiquetado = df_sentiment_num.copy()
col_quitar = ['sentimiento']
set_etiquetado=set_etiquetado.drop(col_quitar, axis=1)
set_etiquetado.shape
set_etiquetado.head(10)

,review_body,positividad
61451,lasted day light work,0.0000
261883,like safari series lamy writes wish send nib e...,0.9273
51718,good price fast shipping,0.4404
176180,batteries work advertised delivered time great...,0.6249
792026,home bad quality,-0.5423
299284,ok,0.2960
654322,little slow compared laser printer price compl...,0.7184
37340,arrvied time expected,0.0000
298849,worked refill work second time ran bummed,0.0000
771477,recorder greatbr br ordered recorder based rea...,0.9870


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Crear un tokenizer
tokenizer = Tokenizer()

# Ajustar el tokenizer al texto de las reseñas
tokenizer.fit_on_texts(set_etiquetado['review_body'])

# Generar las secuencias numéricas para cada reseña y agregarlas a una nueva columna 'secuencia_num'
set_etiquetado['secuencia_num'] = tokenizer.texts_to_sequences(set_etiquetado['review_body'])

# Obtener la longitud máxima de las secuencias
max_length = max(len(seq) for seq in set_etiquetado['secuencia_num'])
print(max_length)

# Ajustar todas las secuencias a la misma longitud
set_etiquetado['secuencia_num'] = pad_sequences(set_etiquetado['secuencia_num'], maxlen=max_length, padding='post')

921


In [ ]:
set_etiquetado.head(10)

,review_body,positividad,secuencia_num
61451,lasted day light work,0.0000,422
261883,like safari series lamy writes wish send nib e...,0.9273,7
51718,good price fast shipping,0.4404,2
176180,batteries work advertised delivered time great...,0.6249,258
792026,home bad quality,-0.5423,118
299284,ok,0.2960,149
654322,little slow compared laser printer price compl...,0.7184,24
37340,arrvied time expected,0.0000,14526
298849,worked refill work second time ran bummed,0.0000,33
771477,recorder greatbr br ordered recorder based rea...,0.9870,657


In [ ]:
review = set_etiquetado['review_body'].iloc[1010]
secuencia_completa = set_etiquetado['secuencia_num'].iloc[1010]
sentimiento_ejemplo = set_etiquetado['positividad'].iloc[1010]
print(review, ": ", secuencia_completa, ": ", sentimiento_ejemplo)


refrigerator time gotten lot compliments extra isnt white markers coming cleanly helps family :  2277 :  0.6486


In [ ]:
#Creamos DF set con secuencias numéricas
set_final = set_etiquetado.copy()
#set_final['secuencia_num'] = padded_sequences
set_final.drop('review_body', axis=1, inplace=True)



---
DIVISIÓN SET DE ENTRENAMIENTO Y VALIDACIÓN
---



In [ ]:
from sklearn.model_selection import train_test_split

X = set_etiquetado['secuencia_num']  # Secuencias numéricas de las reseñas
y = set_etiquetado['positividad']  # Variable objetivo (sentimiento)

# División de los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificación de las formas de los conjuntos resultantes
print("Forma del conjunto de entrenamiento:", X_train.shape)
print("Forma del conjunto de validación:", X_val.shape)

Forma del conjunto de entrenamiento: (39999,)
Forma del conjunto de validación: (10000,)


In [ ]:
print(type(X))

<class 'pandas.core.series.Series'>


In [ ]:
set_final.head(30)

,positividad,secuencia_num
61451,0.0000,422
261883,0.9273,7
51718,0.4404,2
176180,0.6249,258
792026,-0.5423,118
299284,0.2960,149
654322,0.7184,24
37340,0.0000,14526
298849,0.0000,33
771477,0.9870,657


# RED NEURONAL

In [ ]:
from keras.preprocessing.text import Tokenizer

# Obtener el tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

embedding_dim = 100
max_length = 921

# Definir la arquitectura de la red neuronal
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))  # Capa de incrustación de palabras
model.add(LSTM(units=100))  # Capa LSTM con 100 unidades
model.add(Dense(1, activation='relu'))  # Capa de salida con función de activación relu

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 921, 100)          2983500   
                                                                 
 lstm_9 (LSTM)               (None, 100)               80400     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 3,064,001
Trainable params: 3,064,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.losses import BinaryCrossentropy, MeanSquaredError

# Definir funciones de "loss"
sentiment_loss_fn = BinaryCrossentropy(from_logits=False)
score_loss_fn = MeanSquaredError()

# Compilar
model.compile(loss=[sentiment_loss_fn, score_loss_fn], optimizer='sgd', metrics=[tf.keras.metrics.Accuracy()])

# Entrenar modelo
model.fit(X_train, y_train, batch_size=100, epochs=10, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_val, y_val)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/10
400/400 [==============================] - 18s 40ms/step - loss: 8.4701 - accuracy: 2.0001e-04 - val_loss: 8.4059 - val_accuracy: 0.0000e+00
Epoch 2/10
400/400 [==============================] - 10s 26ms/step - loss: 8.4765 - accuracy: 0.0000e+00 - val_loss: 8.4059 - val_accuracy: 0.0000e+00
Epoch 3/10
400/400 [==============================] - 10s 26ms/step - loss: 8.4765 - accuracy: 0.0000e+00 - val_loss: 8.4059 - val_accuracy: 0.0000e+00
Epoch 4/10
400/400 [==============================] - 10s 25ms/step - loss: 8.4765 - accuracy: 0.0000e+00 - val_loss: 8.4059 - val_accuracy: 0.0000e+00
Epoch 5/10
400/400 [==============================] - 10s 26ms/step - loss: 8.4765 - accuracy: 0.0000e+00 - val_loss: 8.4059 - val_accuracy: 0.0000e+00
Epoch 6/10
400/400 [==============================] - 10s 26ms/step - loss: 8.4765 - accuracy: 0.0000e+00 - val_loss: 8.4059 - val_accuracy: 0.0000e+00
Epoch 7/10
400/400 [==============================] - 10s 26ms/step - loss: 8.4765 - acc

In [ ]:
predictions = model.predict(X_val)
np.argmax(predictions[9999])

313/313 [==============================] - 1s 2ms/step


0

# NORMALIZACIÓN DE VALORES NUMÉRICOS


In [ ]:
#IGNORAR POR AHORA
#   solo se normalizará el set de entrenamiento IGNORAR POR AHORA
#   Al haber rangos diferentes en el rating y el número total de votos, se normalizarán a una escala de -1 a 1 para evitar que una característica predomine más que otras
#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#data['star_rating'] = scaler.fit_transform(data['star_rating'].values.reshape(-1, 1))
#data['total_votes'] = scaler.fit_transform(data['total_votes'].values.reshape(-1, 1))
#data['helpful_votes'] = scaler.fit_transform(data['helpful_votes'].values.reshape(-1, 1))